<a href="https://colab.research.google.com/github/qdrptd/RNE/blob/main/comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [3]:
from selenium import webdriver
import time
from openpyxl import Workbook
import pandas as pd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [4]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
 
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
 
#해당 url로 이동
search = 'metaverse'
url = "https://www.youtube.com/results?search_query=" + search
driver.get(url)

In [5]:
driver.execute_script("window.scrollTo(0,1000)")
time.sleep(3)

# 페이지 끝까지 스크롤(밑에서 베낌)
last_height = driver.execute_script("return document.documentElement.scrollHeight")
print(last_height)
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

    if new_height > 100000:
      break

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
video_list = soup.select("a#video-title")

id, titles = [], []
for video in video_list: 
  id.append(video['href'])
  titles.append(video['title'])

5611


In [11]:
views, channels, subs = [], [], []

channel_id_list = soup.select('div#channel-info > a')
views_list = soup.select("div#metadata > div > span:nth-of-type(1)")
time_list = soup.select("div#metadata > div > span:nth-of-type(2)")

for channel_id in channel_id_list:
  channels.append(channel_id['href'])
  # channel_url = 
for v in views_list:
  views.append(v.text)
print(len(id), len(titles), len(views), len(channels), len(subs))
pd_data = {"아이디" : id, "제목" : titles, "조회수": views, "채널": channels}#, "구독자": subs}
youtube_pd = pd.DataFrame(pd_data)

447 447 447 447 0


In [12]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 27.6 MB/s 


In [13]:
youtube_pd.to_excel('youtube.xlsx')

In [15]:
wb = Workbook(write_only=True)
ws = wb.create_sheet()
for video_id in id:
  url = "https://www.youtube.com/watch?v=" + video_id
  driver.get(url)
  driver.implicitly_wait(3)

  time.sleep(1.5)

  driver.execute_script("window.scrollTo(0,1000)")
  time.sleep(3)

  # 페이지 끝까지 스크롤
  last_height = driver.execute_script("return document.documentElement.scrollHeight")

  while True:
      driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
      time.sleep(1.5)

      new_height = driver.execute_script("return document.documentElement.scrollHeight")
      if new_height == last_height:
          break
      last_height = new_height

  time.sleep(1.5)

  # 팝업 닫기
  try:
      driver.find_element_by_css_selector("#dismiss-button > a").click()
  except:
      pass

  # 대댓글 모두 열기
  buttons = driver.find_elements_by_css_selector("#more-replies > a")

  time.sleep(1.5)

  for button in buttons:
      button.send_keys(Keys.ENTER)
      time.sleep(1.5)
      button.click()

  time.sleep(1.5)

  # 정보 추출하기
  html_source = driver.page_source
  soup = BeautifulSoup(html_source, 'html.parser')

  id_list = soup.select("div#header-author > h3 > #author-text > span")
  comment_list = soup.select("yt-formatted-string#content-text")

  id_final = []
  comment_final = []

  for i in range(len(comment_list)):
      temp_id = id_list[i].text
      temp_id = temp_id.replace('\n', '')
      temp_id = temp_id.replace('\t', '')
      temp_id = temp_id.replace('    ', '')
      id_final.append(temp_id)

      temp_comment = comment_list[i].text
      temp_comment = temp_comment.replace('\n', '')
      temp_comment = temp_comment.replace('\t', '')
      temp_comment = temp_comment.replace('    ', '')
      comment_final.append(temp_comment)

  pd_data = {"아이디" : id_final , "댓글 내용" : comment_final}
  youtube_pd = pd.DataFrame(pd_data)

  youtube_pd.to_excel('comments.xlsx')
  print("종료")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead


종료
종료


ProtocolError: ignored